<a href="https://colab.research.google.com/github/cheolhakja/fine-dust-prediction/blob/main/graduation_%EC%A0%84%EC%B2%98%EB%A6%AC_%ED%95%99%EC%8A%B5%EC%85%8B_%EB%A7%8C%EB%93%A4%EA%B8%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#csv 불러오기
from google.colab import drive
drive.mount('/content/drive')

import pandas as pd

weather_path = '/content/drive/MyDrive/graduation_project/weather_202506_all.csv'
air_path = '/content/drive/MyDrive/graduation_project/air_202506.csv'

df_weather = pd.read_csv(weather_path)
df_air_june = pd.read_csv(air_path, encoding='utf-8-sig')
weather_meta = pd.read_csv('/content/drive/MyDrive/graduation_project/aws_station_metadata.csv', encoding='euc-kr')
air_meta = pd.read_csv('/content/drive/MyDrive/graduation_project/station_info.csv')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#강수12h 계산
from tqdm import tqdm

rain_dfs = []

for site in tqdm(df_weather['지점'].unique()):
    df_site = df_weather[df_weather['지점'] == site].sort_values('datetime').copy()

    df_site['강수12h'] = (
        df_site['1분 강수량(mm)']
        .rolling(window=720, min_periods=720)
        .sum()
    )

    rain_dfs.append(df_site)

df_weather = pd.concat(rain_dfs)
df_weather = df_weather.dropna(subset=['강수12h'])

100%|██████████| 89/89 [00:02<00:00, 38.30it/s]


In [ ]:

# 정각인 행만 필터링 (분=0, 초=0)
df_weather['datetime'] = pd.to_datetime(df_weather['datetime'])

df_weather = df_weather[(df_weather['datetime'].dt.minute == 0) & (df_weather['datetime'].dt.second == 0)].copy()

#df_weather.head(30)

In [ ]:
#✅ 최종적으로 남길 열 목록

columns_to_keep_weather = [
    'datetime', '풍속(m/s)', '풍향(deg)', '기온(°C)', '지점', '강수12h'

    #측정소 정보 관련 칼럼 추가 (여기에)

]

columns_to_keep_air = [
    'so2Value', 'coValue', 'o3Value', 'no2Value', 'pm10Value', 'pm25Value','stationName','datetime'

    #측정소 정보 관련 칼럼 추가 (여기에)
]

df_weather = df_weather[columns_to_keep_weather].copy()
df_air_june = df_air_june[columns_to_keep_air].copy()

In [ ]:
df_weather.head(10)

,datetime,풍속(m/s),풍향(deg),기온(°C),지점,강수12h
719,2025-06-01 12:00:00,1.1,341.4,28.0,400,0.0
779,2025-06-01 13:00:00,1.0,132.5,28.9,400,0.0
839,2025-06-01 14:00:00,0.5,329.4,29.2,400,0.0
899,2025-06-01 15:00:00,1.3,282.6,29.0,400,0.0
959,2025-06-01 16:00:00,3.8,251.8,28.6,400,0.0
1019,2025-06-01 17:00:00,2.9,257.3,27.6,400,0.0
1079,2025-06-01 18:00:00,2.7,216.1,26.5,400,0.0
1139,2025-06-01 19:00:00,1.0,262.4,25.3,400,0.0
1199,2025-06-01 20:00:00,1.2,292.9,24.5,400,0.0
1259,2025-06-01 21:00:00,0.9,289.9,23.6,400,0.0


In [ ]:
df_air_june.head(10)

,so2Value,coValue,o3Value,no2Value,pm10Value,pm25Value,stationName,datetime
0,0.0019,0.18,0.0097,0.0115,7,7,중구,2025-06-30 23:00:00
1,0.0022,0.19,0.0121,0.0108,7,4,중구,2025-06-30 22:00:00
2,0.003,0.19,0.0162,0.0114,11,8,중구,2025-06-30 21:00:00
3,0.0026,0.2,0.0206,0.012,11,9,중구,2025-06-30 20:00:00
4,0.0031,0.21,0.0265,0.013,9,8,중구,2025-06-30 19:00:00
5,0.0029,0.19,0.0302,0.0116,11,8,중구,2025-06-30 18:00:00
6,0.0032,0.18,0.0369,0.0106,12,7,중구,2025-06-30 17:00:00
7,0.0025,0.18,0.0377,0.0105,12,8,중구,2025-06-30 16:00:00
8,0.0025,0.2,0.0337,0.0109,15,9,중구,2025-06-30 15:00:00
9,0.0027,0.2,0.0342,0.0098,15,11,중구,2025-06-30 14:00:00


In [ ]:
##################################################
# 이 아래로 메타데이터 전처리
##################################################

In [ ]:
# 종료일이 결측치(NaN)인 행만 추출
weather_meta = weather_meta[weather_meta['종료일'].isna()].copy()

#사용중인 방재 측정소중 결측치를 경기도 부천시 원미구로 채워넣기 (결측치가 하나밖에 없어서 이게 가능한거)
weather_meta['지점주소'] = weather_meta['지점주소'].fillna('경기도 부천시 원미구')

#경기도 시/군 리스트
gyeonggi_cities = [
    '수원시', '고양시', '용인시', '화성시', '성남시', '의정부시', '안양시', '부천시', '광명시',
    '평택시', '동두천시', '안산시', '과천시', '구리시', '남양주시', '오산시', '시흥시', '군포시',
    '의왕시', '하남시', '파주시', '이천시', '안성시', '김포시', '광주시', '양주시', '포천시',
    '여주시', '연천군', '가평군', '양평군'
]

In [ ]:
#✅ 최종적으로 남길 열 목록

columns_to_keep_weathermeta = [
    '지점','지점명','지점주소'



]

columns_to_keep_airmeta = [
    'stationName', 'addr', 'dmX', 'dmY'

]

weather_meta = weather_meta[columns_to_keep_weathermeta].copy()
air_meta = air_meta[columns_to_keep_airmeta].copy()

In [ ]:
weather_meta.head(3)

,지점,지점명,지점주소
0,116,관악(레),경기도 과천시 중앙동
3,309,판문점,경기도 파주시 군내면 조산리
4,326,용문산,경기도 양평군 용문면 중원리


In [ ]:
air_meta.head(3)

,stationName,addr,dmX,dmY
0,중구,서울 중구 덕수궁길 15 시청서소문별관 3동,37.564639,126.975961
1,한강대로,서울 용산구 한강대로 405 (서울역 앞),37.549389,126.971519
2,종로구,"서울 종로구 종로35가길 19 종로5,6가 동 주민센터",37.572025,127.005028


In [ ]:
############################################################################################
# air meta 에 가까운 측정소 5개 추가
############################################################################################
from geopy.distance import geodesic
import pandas as pd

df = air_meta.copy()

nearest_5_list = []

for idx, row in df.iterrows():
    lat, lon = row['dmX'], row['dmY']

    def wgs84_distance(x):
        return geodesic((lat, lon), (x['dmX'], x['dmY'])).km

    df['distance'] = df.apply(wgs84_distance, axis=1)
    nearest_5 = df[df.index != idx].nsmallest(5, 'distance')['stationName'].tolist()
    nearest_5_list.append(nearest_5)

# 리스트를 DataFrame으로 변환, 컬럼명 지정
nearest_5_df = pd.DataFrame(nearest_5_list, columns=['V1_stationName', 'V2_stationName', 'V3_stationName', 'V4_stationName', 'V5_stationName'])

# 원본 df와 합치기
df = pd.concat([df.reset_index(drop=True), nearest_5_df], axis=1)

air_meta = df.copy()


In [ ]:
#air meta에서 addr의 두번째 단어가 구/시/군 으로 끝나는지 체크해보자
second_words_airmeta = air_meta['addr'].apply(lambda x: x.split()[1] if len(x.split()) > 1 else '').tolist()

ends_with_gu_si_gun = [word.endswith(('구', '시', '군')) for word in second_words_airmeta]

all_ends_correctly = all(ends_with_gu_si_gun)
print(all_ends_correctly)  # True 면 모두 맞음



True


In [ ]:
#air meta에 구 시 군 칼럼 추가 -> air_meta 데이터프레임에 방재기상 지점코드 매칭하려고
air_meta['구/시/군'] = air_meta['addr'].apply(lambda x: x.split()[1] if len(x.split()) > 1 else '')



In [ ]:
# air_meta 데이터프레임에 방재기상 지점코드 매칭
import numpy as np

# 새로운 컬럼 생성
air_meta['aws지점코드'] = np.nan

# 루프 돌면서 매칭
for i, air_row in air_meta.iterrows():
    target_city = air_row['구/시/군']

    if pd.isna(target_city):
        continue  # 시군이 비어있으면 skip

    for _, weather_row in weather_meta.iterrows():
        # 두 번째 단어 추출
        weather_second_word = weather_row['지점주소'].split()[1] if len(weather_row['지점주소'].split()) > 1 else ''

        # 포함 관계 확인
        if target_city in weather_second_word:
            air_meta.at[i, 'aws지점코드'] = weather_row['지점']
            break  # 첫 매칭에서 바로 종료

In [ ]:
# air meta와 weather meta를 합친 뒤,  aws지점코드가 NaN인거 확인하기
num_nan = air_meta['aws지점코드'].isna().sum()
print(num_nan) #수원시8 해운대1 동두천시1 종로구2

#NaN제거
air_meta = air_meta.dropna(subset=['aws지점코드'])


12


In [ ]:
# df_air_june 과 air_meta 데이터프레임을 stationName을 기준으로 합치기
# df_air_june과 df_weather을 datetime을  기준으로 합치기
# 가까운 5개 측정소 구하기

In [ ]:
#어떤 행이 left join이 안되지? -> aws지점코드가 없는 곳 수원시8 해운대1 동두천시1 종로구2
set_air_june = set(df_air_june['stationName'].unique())
set_air_meta = set(air_meta['stationName'].unique())

not_in_air_meta = set_air_june - set_air_meta
print(not_in_air_meta)


{'우동', '경수대로(동수원)', '천천동', '인계동', '신풍동', '광교동', '영통동', '호매실', '종로구', '고색동', '보산동', '종로'}


In [ ]:
df_air_june = pd.merge(df_air_june, air_meta, on='stationName', how='left')


In [ ]:
# 전체 결측치 개수 확인
print(df_air_june.isna().sum())

# 특정 컬럼 결측치만 보고 싶으면 예를 들어 'aws지점코드'
print(df_air_june['aws지점코드'].isna().sum())

# 결측치가 있는 행만 보고 싶으면
print(df_air_june[df_air_june.isna().any(axis=1)])

#전체행
print(df_air_june.shape[0])

#aws 지점코드 없는 곳 수원시8 해운대1 동두천시1 종로구2 지우자
df_air_june.dropna(subset=['aws지점코드'], inplace=True)

# 전체 결측치 개수 확인
print(df_air_june.isna().sum())

# 특정 컬럼 결측치만 보고 싶으면 예를 들어 'aws지점코드'
print(df_air_june['aws지점코드'].isna().sum())

# 결측치가 있는 행만 보고 싶으면
print(df_air_june[df_air_june.isna().any(axis=1)])


so2Value             0
coValue              0
o3Value              0
no2Value             0
pm10Value            0
pm25Value            0
stationName          0
datetime             0
addr              8257
dmX               8257
dmY               8257
distance          8257
V1_stationName    8257
V2_stationName    8257
V3_stationName    8257
V4_stationName    8257
V5_stationName    8257
구/시/군             8257
aws지점코드           8257
dtype: int64
8257
      so2Value coValue o3Value no2Value pm10Value pm25Value stationName  \
1380    0.0025    0.18  0.0101   0.0103         6         5         종로구   
1381    0.0026    0.18  0.0116     0.01         7         8         종로구   
1382    0.0026    0.24  0.0139   0.0126        13        11         종로구   
1383     0.003    0.23  0.0181   0.0132        11        11         종로구   
1384    0.0026    0.22  0.0273   0.0117        13         8         종로구   
...        ...     ...     ...      ...       ...       ...         ...   
97055   0.0018    0.

In [ ]:
df_weather.rename(columns={'지점': 'aws지점코드'}, inplace=True)
# df_weather.head(10)

In [ ]:
# 1. 병합 (left join) + indicator=True로 매칭 상태 표시
import pandas as pd

#aws 지점코드 소수형을 정수형으로
df_air_june['aws지점코드'] = df_air_june['aws지점코드'].astype('Int64')  # nullable integer
df_weather['aws지점코드'] = df_weather['aws지점코드'].astype('Int64')

# datetime 타입 변환
df_air_june['datetime'] = pd.to_datetime(df_air_june['datetime'])
df_weather['datetime'] = pd.to_datetime(df_weather['datetime'])

# 병합
df_air_june = pd.merge(
    df_air_june,
    df_weather,
    on=['datetime', 'aws지점코드'],
    how='left',
    indicator=True
)

# 매칭 실패 행 확인
unmatched = df_air_june[df_air_june['_merge'] == 'left_only']
print(f"매칭 실패 행 개수: {len(unmatched)}")
print(unmatched.head())

매칭 실패 행 개수: 17711
  so2Value coValue o3Value no2Value pm10Value pm25Value stationName  \
3   0.0026     0.2  0.0206    0.012        11         9          중구   
4   0.0031    0.21  0.0265    0.013         9         8          중구   
5   0.0029    0.19  0.0302   0.0116        11         8          중구   
6   0.0032    0.18  0.0369   0.0106        12         7          중구   
7   0.0025    0.18  0.0377   0.0105        12         8          중구   

             datetime                      addr        dmX  ...  \
3 2025-06-30 20:00:00  서울 중구 덕수궁길 15 시청서소문별관 3동  37.564639  ...   
4 2025-06-30 19:00:00  서울 중구 덕수궁길 15 시청서소문별관 3동  37.564639  ...   
5 2025-06-30 18:00:00  서울 중구 덕수궁길 15 시청서소문별관 3동  37.564639  ...   
6 2025-06-30 17:00:00  서울 중구 덕수궁길 15 시청서소문별관 3동  37.564639  ...   
7 2025-06-30 16:00:00  서울 중구 덕수궁길 15 시청서소문별관 3동  37.564639  ...   

   V3_stationName  V4_stationName V5_stationName 구/시/군 aws지점코드 풍속(m/s)  \
3            청계천로             종로구            신촌로    중구     419     NaN   
4   

In [ ]:
failed_keys = unmatched[['datetime', 'aws지점코드']].drop_duplicates()
print(failed_keys.head())



             datetime  aws지점코드
3 2025-06-30 20:00:00      419
4 2025-06-30 19:00:00      419
5 2025-06-30 18:00:00      419
6 2025-06-30 17:00:00      419
7 2025-06-30 16:00:00      419


In [ ]:
for idx, row in failed_keys.iterrows():
    dt = row['datetime']
    code = row['aws지점코드']
    exists = ((df_weather['datetime'] == dt) & (df_weather['aws지점코드'] == code)).any()
    print(f"datetime: {dt}, aws지점코드: {code}, 존재여부: {exists}")


스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
datetime: 2025-06-17 03:00:00, aws지점코드: 412, 존재여부: False
datetime: 2025-06-17 02:00:00, aws지점코드: 412, 존재여부: False
datetime: 2025-06-17 01:00:00, aws지점코드: 412, 존재여부: False
datetime: 2025-06-16 23:00:00, aws지점코드: 412, 존재여부: False
datetime: 2025-06-16 22:00:00, aws지점코드: 412, 존재여부: False
datetime: 2025-06-16 21:00:00, aws지점코드: 412, 존재여부: False
datetime: 2025-06-16 20:00:00, aws지점코드: 412, 존재여부: False
datetime: 2025-06-16 19:00:00, aws지점코드: 412, 존재여부: False
datetime: 2025-06-16 18:00:00, aws지점코드: 412, 존재여부: False
datetime: 2025-06-16 17:00:00, aws지점코드: 412, 존재여부: False
datetime: 2025-06-01 11:00:00, aws지점코드: 412, 존재여부: False
datetime: 2025-06-01 10:00:00, aws지점코드: 412, 존재여부: False
datetime: 2025-06-01 09:00:00, aws지점코드: 412, 존재여부: False
datetime: 2025-06-01 08:00:00, aws지점코드: 412, 존재여부: False
datetime: 2025-06-01 07:00:00, aws지점코드: 412, 존재여부: False
datetime: 2025-06-01 06:00:00, aws지점코드: 412, 존재여부: False
datetime: 2025-06-01 05:00:00, aws지점코드: 412, 존재여부: F

In [ ]:
print(df_weather[df_weather['aws지점코드'] == 412]['datetime'].min())
print(df_weather[df_weather['aws지점코드'] == 412]['datetime'].max())
print(df_weather[df_weather['aws지점코드'] == 412].shape[0])


2025-06-01 12:00:00
2025-07-01 00:00:00
673


In [ ]:
print(df_weather['datetime'].dt.minute.unique())
print(df_weather['datetime'].dt.second.unique())


[0]
[0]


In [ ]:
failed_codes = unmatched['aws지점코드'].dropna().unique().tolist()
print(f"매칭 실패한 aws지점코드 리스트: {failed_codes}")

for code in failed_codes:
    count = (df_air_june['aws지점코드'] == code).sum()
    print(f"aws지점코드 {code}가 df_air_june에 {count}개 있습니다.")


매칭 실패한 aws지점코드 리스트: [419, 415, 413, 421, 409, 408, 414, 406, 416, 412, 411, 404, 423, 418, 410, 425, 400, 401, 403, 402, 417, 424, 405, 407, 433, 514, 116, 569, 445, 367, 541, 551, 309, 450, 546, 371, 533, 504, 427, 438, 446, 428, 515, 495, 548, 499, 505, 326, 572, 532, 377, 376]
aws지점코드 419가 df_air_june에 1380개 있습니다.
aws지점코드 415가 df_air_june에 1380개 있습니다.
aws지점코드 413가 df_air_june에 690개 있습니다.
aws지점코드 421가 df_air_june에 1380개 있습니다.
aws지점코드 409가 df_air_june에 690개 있습니다.
aws지점코드 408가 df_air_june에 1380개 있습니다.
aws지점코드 414가 df_air_june에 1380개 있습니다.
aws지점코드 406가 df_air_june에 690개 있습니다.
aws지점코드 416가 df_air_june에 690개 있습니다.
aws지점코드 412가 df_air_june에 690개 있습니다.
aws지점코드 411가 df_air_june에 1380개 있습니다.
aws지점코드 404가 df_air_june에 1380개 있습니다.
aws지점코드 423가 df_air_june에 690개 있습니다.
aws지점코드 418가 df_air_june에 1380개 있습니다.
aws지점코드 410가 df_air_june에 1380개 있습니다.
aws지점코드 425가 df_air_june에 690개 있습니다.
aws지점코드 400가 df_air_june에 2064개 있습니다.
aws지점코드 401가 df_air_june에 1380개 있습니다.
aws지점코드 403가 df_air_june에 690개 있습니다.
aws지점

In [ ]:
df_air_june = df_air_june[df_air_june['_merge'] == 'both'].copy()
df_air_june.drop(columns=['_merge'], inplace=True)


In [ ]:
# df_air_june.iloc[:50]

In [ ]:
# #V1 - V5의 대기질 데이터 추가
# import numpy as np

# pollution_df = df_air_june  # 대기질 데이터프레임

# pollution_dict = {
#     (row['stationName'], row['datetime']): {
#         'so2Value': row['so2Value'],
#         'coValue': row['coValue'],
#         'no2Value': row['no2Value'],
#         'o3Value': row['o3Value']
#     }
#     for idx, row in pollution_df.iterrows()
# }

# def get_pollution_value(station, dt, key):
#     return pollution_dict.get((station, dt), {}).get(key, np.nan)

# for i in range(1, 6):
#     df_air_june[f'V{i}_so2Value'] = df_air_june.apply(lambda r: get_pollution_value(r[f'V{i}_stationName'], r['datetime'], 'so2Value'), axis=1)
#     df_air_june[f'V{i}_coValue'] = df_air_june.apply(lambda r: get_pollution_value(r[f'V{i}_stationName'], r['datetime'], 'coValue'), axis=1)
#     df_air_june[f'V{i}_no2Value'] = df_air_june.apply(lambda r: get_pollution_value(r[f'V{i}_stationName'], r['datetime'], 'no2Value'), axis=1)
#     df_air_june[f'V{i}_o3Value'] = df_air_june.apply(lambda r: get_pollution_value(r[f'V{i}_stationName'], r['datetime'], 'o3Value'), axis=1)


In [ ]:
import numpy as np

# 대기질 + 기상 통합 데이터프레임 v1 - v5 추가
merged_df = df_air_june

# 👉 딕셔너리 구성: stationName + datetime 기준
merged_dict = {
    (row['stationName'], row['datetime']): {
        'so2Value': row['so2Value'],
        'coValue': row['coValue'],
        'no2Value': row['no2Value'],
        'o3Value': row['o3Value'],
        'temp': row['기온(°C)'],
        'wind': row['풍속(m/s)'],
        'rain12h': row['강수12h'],
        'pm25Value': row['pm25Value'],
        'pm10Value': row['pm10Value']
    }
    for idx, row in merged_df.iterrows()
}

# 값 조회 함수
def get_value(station, dt, key):
    return merged_dict.get((station, dt), {}).get(key, np.nan)

# 👉 V1~V5 반복
for i in range(1, 6):
    df_air_june[f'V{i}_so2Value'] = df_air_june.apply(lambda r: get_value(r[f'V{i}_stationName'], r['datetime'], 'so2Value'), axis=1)
    df_air_june[f'V{i}_coValue'] = df_air_june.apply(lambda r: get_value(r[f'V{i}_stationName'], r['datetime'], 'coValue'), axis=1)
    df_air_june[f'V{i}_no2Value'] = df_air_june.apply(lambda r: get_value(r[f'V{i}_stationName'], r['datetime'], 'no2Value'), axis=1)
    df_air_june[f'V{i}_o3Value'] = df_air_june.apply(lambda r: get_value(r[f'V{i}_stationName'], r['datetime'], 'o3Value'), axis=1)
    df_air_june[f'V{i}_pm25Value'] = df_air_june.apply(lambda r: get_value(r[f'V{i}_stationName'], r['datetime'], 'pm25Value'), axis=1)
    df_air_june[f'V{i}_pm10Value'] = df_air_june.apply(lambda r: get_value(r[f'V{i}_stationName'], r['datetime'], 'pm10Value'), axis=1)

    # 기상 변수도 같이 추가
    df_air_june[f'V{i}_temp'] = df_air_june.apply(lambda r: get_value(r[f'V{i}_stationName'], r['datetime'], 'temp'), axis=1)
    df_air_june[f'V{i}_wind'] = df_air_june.apply(lambda r: get_value(r[f'V{i}_stationName'], r['datetime'], 'wind'), axis=1)
    df_air_june[f'V{i}_rain12h'] = df_air_june.apply(lambda r: get_value(r[f'V{i}_stationName'], r['datetime'], 'rain12h'), axis=1)


In [ ]:
for i in range(1, 6):
    for pollutant in ['so2Value', 'coValue', 'no2Value', 'o3Value']:
        col = f'V{i}_{pollutant}'
        missing_count = df_air_june[col].isna().sum()
        print(f"{col} 결측치 개수: {missing_count}")


V1_so2Value 결측치 개수: 5717
V1_coValue 결측치 개수: 5717
V1_no2Value 결측치 개수: 5717
V1_o3Value 결측치 개수: 5717
V2_so2Value 결측치 개수: 9335
V2_coValue 결측치 개수: 9335
V2_no2Value 결측치 개수: 9335
V2_o3Value 결측치 개수: 9335
V3_so2Value 결측치 개수: 9423
V3_coValue 결측치 개수: 9423
V3_no2Value 결측치 개수: 9423
V3_o3Value 결측치 개수: 9423
V4_so2Value 결측치 개수: 15162
V4_coValue 결측치 개수: 15162
V4_no2Value 결측치 개수: 15162
V4_o3Value 결측치 개수: 15162
V5_so2Value 결측치 개수: 13013
V5_coValue 결측치 개수: 13013
V5_no2Value 결측치 개수: 13013
V5_o3Value 결측치 개수: 13013


In [ ]:
#결측 확인
missing_v4 = df_air_june[df_air_june['V4_so2Value'].isna()]
print(missing_v4[['V4_stationName', 'datetime']].drop_duplicates().head(20))


   V4_stationName            datetime
0             종로구 2025-06-30 23:00:00
1             종로구 2025-06-30 22:00:00
2             종로구 2025-06-30 21:00:00
15            종로구 2025-06-30 08:00:00
16            종로구 2025-06-30 07:00:00
17            종로구 2025-06-30 06:00:00
18            종로구 2025-06-30 05:00:00
19            종로구 2025-06-30 04:00:00
20            종로구 2025-06-30 03:00:00
21            종로구 2025-06-30 02:00:00
22            종로구 2025-06-30 01:00:00
23            종로구 2025-06-29 23:00:00
24            종로구 2025-06-29 22:00:00
25            종로구 2025-06-29 21:00:00
26            종로구 2025-06-29 20:00:00
27            종로구 2025-06-29 19:00:00
28            종로구 2025-06-29 18:00:00
29            종로구 2025-06-29 17:00:00
30            종로구 2025-06-29 16:00:00
31            종로구 2025-06-29 15:00:00


In [ ]:
len(df_air_june)

88262

In [ ]:
cols_to_check = []

for i in range(1, 6):
    cols_to_check.extend([
        f'V{i}_so2Value',
        f'V{i}_coValue',
        f'V{i}_no2Value',
        f'V{i}_o3Value',
        f'V{i}_temp',
        f'V{i}_wind',
        f'V{i}_rain12h',
        f'V{i}_pm25Value',
        f'V{i}_pm10Value'
    ])

df_air_june_clean = df_air_june.dropna(subset=cols_to_check)


print(f"결측치 제거 전 행 개수: {len(df_air_june)}")
print(f"결측치 제거 후 행 개수: {len(df_air_june_clean)}")


결측치 제거 전 행 개수: 88262
결측치 제거 후 행 개수: 55748


In [ ]:
df_air_june_clean = df_air_june_clean.dropna()


In [ ]:
null_counts = df_air_june_clean.isnull().sum()
null_counts = null_counts[null_counts > 0]  # 결측치가 있는 컬럼만 추림

if null_counts.empty:
    print("✅ df_air_june_clean에는 결측치가 있는 컬럼이 전혀 없습니다!")
else:
    print("❌ 다음 컬럼에 결측치가 남아있습니다:")
    print(null_counts)


✅ df_air_june_clean에는 결측치가 있는 컬럼이 전혀 없습니다!


In [ ]:
print(df_air_june_clean.columns.tolist())


['so2Value', 'coValue', 'o3Value', 'no2Value', 'pm10Value', 'pm25Value', 'stationName', 'datetime', 'addr', 'dmX', 'dmY', 'distance', 'V1_stationName', 'V2_stationName', 'V3_stationName', 'V4_stationName', 'V5_stationName', '구/시/군', 'aws지점코드', '풍속(m/s)', '풍향(deg)', '기온(°C)', '강수12h', 'V1_so2Value', 'V1_coValue', 'V1_no2Value', 'V1_o3Value', 'V1_pm25Value', 'V1_pm10Value', 'V1_temp', 'V1_wind', 'V1_rain12h', 'V2_so2Value', 'V2_coValue', 'V2_no2Value', 'V2_o3Value', 'V2_pm25Value', 'V2_pm10Value', 'V2_temp', 'V2_wind', 'V2_rain12h', 'V3_so2Value', 'V3_coValue', 'V3_no2Value', 'V3_o3Value', 'V3_pm25Value', 'V3_pm10Value', 'V3_temp', 'V3_wind', 'V3_rain12h', 'V4_so2Value', 'V4_coValue', 'V4_no2Value', 'V4_o3Value', 'V4_pm25Value', 'V4_pm10Value', 'V4_temp', 'V4_wind', 'V4_rain12h', 'V5_so2Value', 'V5_coValue', 'V5_no2Value', 'V5_o3Value', 'V5_pm25Value', 'V5_pm10Value', 'V5_temp', 'V5_wind', 'V5_rain12h']


In [ ]:
# 학습 데이터셋 저장
df_air_june_clean.to_csv('/content/drive/MyDrive/graduation_project/train_202506.csv', index=False)


In [ ]:
##################################################
# 이 아래로 본격 학습 데이터셋 만들기 (x-1시 와 x 시의 자)
##################################################

In [ ]:
# # 1단계: 데이터 준비 및 타입 확인
# # datetime 컬럼이 datetime 타입인지 확인하고, 아니면 변환하기
# df_air_june_clean = df_air_june_clean.copy()
# df_air_june_clean['datetime'] = pd.to_datetime(df_air_june_clean['datetime'])


In [ ]:
# # 2단계: 1시간 전 시간 컬럼 만들기
# # 현재 시간에서 1시간 뺀 datetime_minus_1h 컬럼 생성
# df_air_june_clean['datetime_minus_1h'] = df_air_june_clean['datetime'] - pd.Timedelta(hours=1)


In [ ]:
# # 3단계: 자기 자신과 1시간 전 데이터 병합
# # 같은 stationName과 datetime_minus_1h를 기준으로 자기 자신과 내부 조인
# train_df = pd.merge(
#     df_air_june_clean,
#     df_air_june_clean,
#     left_on=['stationName', 'datetime_minus_1h'],
#     right_on=['stationName', 'datetime'],
#     how='inner',
#     suffixes=('', '_lag1h')
# )


In [ ]:
# # 4단계: 사용할 독립변수(입력)와 종속변수(목표) 선택
# # 예시로 주변 5개 측정소의 대기질 변수와 기상 변수 사용
# features = [
#     'V1_so2Value_lag1h', 'V1_coValue_lag1h', 'V1_no2Value_lag1h', 'V1_o3Value_lag1h',
#     'V2_so2Value_lag1h', 'V2_coValue_lag1h', 'V2_no2Value_lag1h', 'V2_o3Value_lag1h',
#     'V3_so2Value_lag1h', 'V3_coValue_lag1h', 'V3_no2Value_lag1h', 'V3_o3Value_lag1h',
#     'V4_so2Value_lag1h', 'V4_coValue_lag1h', 'V4_no2Value_lag1h', 'V4_o3Value_lag1h',
#     'V5_so2Value_lag1h', 'V5_coValue_lag1h', 'V5_no2Value_lag1h', 'V5_o3Value_lag1h',
#     '풍속(m/s)_lag1h', '기온(°C)_lag1h', '강수12h_lag1h'
# ]

# target = ['pm25Value']


In [ ]:
# #5단계: 결측치 제거

# train_df = train_df.dropna(subset=features + target)


In [ ]:
# # 6단계: 최종 학습셋 완성
# # 독립변수와 종속변수를 나누기
# X = train_df[features]
# y = train_df[target]


In [ ]:
# print(X.head())
# print(y.head())
# print(f"학습 데이터 개수: {len(X)}")
